# Computing SFRD

In the present work, we want to update the measurements of star formation rate density from the recent observations. The idea is to use luminosity function (the Schechter function) with the updated parameters from the literature to compute the luminosity density and then using this updated luminosity function to calculate the star formation rate density. In this notebook we try to summarize our work.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

## The Schechter function

The Schechter function (Schechter 1975) is an analytic expression for the luminosity of galaxies which can be written as follows:

$$\phi(L) = \left( \frac{\phi^*}{L^*} \right) \left( \frac{L}{L^*} \right)^\alpha \exp{\left( -\frac{L}{L^*} \right)}$$

This function essentially describes the number of galaxies in the given range of luminosity $L$ to $L+dL$. It is evident from the above equation that it is a three parameter function with parameters $\phi^*$, $L^*$ and $\alpha$. Here, $\alpha$ represents the power-law parameter for the faint-end and $L^*$ is the characteristic luminosity, while $\phi*$ serves as the normalisation constant.